In [55]:
from retriever import Retriever
from app_utils import get_mfa_response
import boto3
import pandas as pd
from langchain_core.documents import Document

In [56]:
mfa_response = get_mfa_response("697648")
if mfa_response is not None:
    session = boto3.Session(aws_access_key_id=mfa_response['Credentials']['AccessKeyId'],
                            aws_secret_access_key=mfa_response['Credentials']['SecretAccessKey'],
                            aws_session_token=mfa_response['Credentials']['SessionToken'])
client = session.client("bedrock-runtime", region_name='eu-west-1')

In [57]:
retriever = Retriever('cohere.embed-multilingual-v3', client=client)

In [58]:
df = pd.read_csv("/home/tommaso/Downloads/export (3).csv", header=0)

In [59]:
documents = []
for index,id,title,text in df.itertuples():
    doc = Document(id=id, page_content=text, metadata={"title": title, "doc_id":id, "source": id.split(".txt")[0]})
    documents.append(doc)

In [60]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Make sure it's less than 2048
    chunk_overlap=100  # Optional: small overlap to preserve context
)

chunks = text_splitter.split_documents(documents)

In [62]:
_ = retriever.vector_store.add_documents(documents=chunks)

In [63]:
retriever.save_vector_store("./reuma_250507.db")

In [54]:
documents

[Document(id='2023 Malattia da deposito calcio pirofosfato.txt--paragraph1.', metadata={'title': ' Criteri classificativi della malattia da deposito di calcio pirofosfato', 'source': '2023 Malattia da deposito calcio pirofosfato'}, page_content='1. Criteri classificativi della malattia da deposito di calcio pirofosfato\nTra i pazienti con dolore, gonfiore o dolorabilità alle articolazioni (criterio di ingresso) i cui sintomi non sono completamente spiegati da una malattia alternativa (criterio di esclusione), la presenza della sindrome del dente incoronato o di cristalli di calcio pirofosfato nel liquido sinoviale sono sufficienti per classificare un paziente come affetto da malattia da deposito di calcio pirofosfato.\n '),
 Document(id='2023 Malattia da deposito calcio pirofosfato.txt--paragraph1.1.', metadata={'title': ' Sviluppo dei criteri classificativi fase 1 e 2', 'source': '2023 Malattia da deposito calcio pirofosfato'}, page_content="1.1. Sviluppo dei criteri classificativi fa